In [1]:
import os
import tempfile

import scanpy as sc
import scvi
import seaborn as sns
import torch

RuntimeError: This version of jaxlib was built using AVX instructions, which your CPU and/or operating system do not support. This error is frequently encountered on macOS when running an x86 Python installation on ARM hardware. In this case, try installing an ARM build of Python. Otherwise, you may be able work around this issue by building jaxlib from source.

In [ ]:
import anndata as ad

In [ ]:
scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)

In [ ]:
sc.set_figure_params(figsize=(6, 6), frameon=False)
sns.set_theme()
torch.set_float32_matmul_precision("high")
save_dir = tempfile.TemporaryDirectory()

%config InlineBackend.print_figure_kwargs={"facecolor": "w"}
%config InlineBackend.figure_format="retina"

In [ ]:
adata = ad.read_h5ad('/Users/apple/Desktop/KB/data/LarryData/Larry_41201_25289_scvi-1.h5ad')
print("adata.X.shape:", adata.X.shape)

In [ ]:
sc.pp.filter_genes(adata, min_counts=3)

In [ ]:
adata.layers["counts"] = adata.X.copy()  # preserve counts
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata  # freeze the state in `.raw`

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    subset=True,
    layer="counts",
    flavor="seurat_v3",
    batch_key=None,
)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata,
    layer="counts",
)

In [ ]:
model = scvi.model.SCVI(adata)

In [ ]:
model 

In [ ]:
model.train()

In [ ]:
model_dir = os.path.join(save_dir.name, "scvi_model")
model.save(model_dir, overwrite=True)

In [ ]:
SCVI_LATENT_KEY = "X_scVI"

latent = model.get_latent_representation()
adata.obsm[SCVI_LATENT_KEY] = latent
latent.shape

In [ ]:
type(latent)

In [ ]:
import numpy as np
np.save('Larry_scvi-1.npy', latent)